In [160]:
from __future__ import division, print_function
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import skfuzzy as fuzz
import pandas as pd
from pandas import DataFrame
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score

In [161]:
colors = ['b', 'orange', 'g', 'r', 'c', 'm', 'y', 'k', 'Brown', 'ForestGreen']
# Directory of the dataset 
filename = 'googl.us.txt'

# Read the file
Prgoo = pd.read_csv(filename,sep=',',index_col='Date')

# Prices is the predict value and initial the independet variable (y)
#prices = Prgoo['Close'].to_numpy()
#initial = Prgoo['Open'].to_numpy()

prices = Prgoo['Close'].tolist()
initial = (Prgoo['Open']).tolist()
print(type(prices))
#Convert to 1d Vector
# prices = np.reshape(prices, (len(prices), 1))
# initial = np.reshape(initial, (len(initial), 1))

Prgoo.head(5)
df = DataFrame(Prgoo,columns=['Close','Open','High','Low','Volume'])

<class 'list'>


In [162]:
cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(
        df, 2000, 2, error=0.005, maxiter=1000, init=None)
print(fpc)



# Set up the loop and plot
# fig1, axes1 = plt.subplots(3, 3, figsize=(8, 8))
# fpcs = []
# for ncenters, ax in enumerate(axes1.reshape(-1), 2):
#     cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(
#         df, ncenters, 2, error=0.005, maxiter=1000, init=None)

#     # Store fpc values for later
#     fpcs.append(fpc)

#     # Plot assigned clusters, for each data point in training set
#     cluster_membership = np.argmax(u, axis=0)
#     for j in range(ncenters):
#         ax.plot(df['Close'], df['Open'], c=colors[j] )

#     # Mark the center of each fuzzy cluster
#     for pt in cntr:
#         ax.plot(pt[0], pt[1], 'rs')

#     ax.set_title('Centers = {0}; FPC = {1:.2f}'.format(ncenters, fpc))
#     ax.axis('off')

# fig1.tight_layout()

0.4404753206806185


In [163]:
# plt.scatter(df['Close'], df['Open'], c='g' , s=50, alpha=0.5)
# plt.scatter(cntr[:, 0], cntr[:, 1], c='red', s=50)

# cluster_membership = np.argmax(u, axis=0) 
# print(cluster_membership)
# fig2, ax2 = plt.subplots()
# ax2.set_title('Trained model')
# for j in range(3):
#     ax2.plot(df['Close'][cluster_membership == j],
#                 df['Open'][cluster_membership == j], 'o',
#              label='series ' + str(j))
# ax2.legend()
u, u0, d, jm, p, fpc = fuzz.cluster.cmeans_predict(
    df, cntr, 2, error=0.005, maxiter=1000)
X=u
y=u.argmax(axis=1)

In [164]:
from fcmeans import FCM
from seaborn import scatterplot as scatter

In [165]:
#fcm = FCM(n_clusters=2)
#fcm.fit(df)

# outputs
#fcm_centers = fcm.centers
#fcm_labels  = fcm.u.argmax(axis=1)
#X=fcm.u.argmax(axis=0)
#y=fcm_labels

#print(X)

xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.33, random_state=42)
print(" %s -> %s" % (len(xtrain), len(ytrain)))

print('----------------------------------------------------------')
print(" %s -> %s" % (len(xtest), len(ytest)))

 1340 -> 1340
----------------------------------------------------------
 660 -> 660


In [166]:
from sklearn.neural_network import MLPClassifier
regressor = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(5, 2), random_state=1)
regressor.fit(xtrain, ytrain)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(5, 2), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [167]:
y_pred=regressor.predict(xtest)
print("Accuracy:",accuracy_score(ytest, y_pred))

Accuracy: 0.9984848484848485


In [168]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
Accuracy_Score = accuracy_score(ytest, y_pred)
Precision_Score = precision_score(ytest, y_pred,  average="macro")
Recall_Score = recall_score(ytest, y_pred,  average="macro")
F1_Score = f1_score(ytest, y_pred,  average="macro")

print('Average Accuracy: %0.2f +/- (%0.1f) %%' % (Accuracy_Score.mean()*100, Accuracy_Score.std()*100))
print('Average Precision: %0.2f +/- (%0.1f) %%' % (Precision_Score.mean()*100, Precision_Score.std()*100))
print('Average Recall: %0.2f +/- (%0.1f) %%' % (Recall_Score.mean()*100, Recall_Score.std()*100))
print('Average F1-Score: %0.2f +/- (%0.1f) %%' % (F1_Score.mean()*100, F1_Score.std()*100))

Average Accuracy: 99.85 +/- (0.0) %
Average Precision: 98.75 +/- (0.0) %
Average Recall: 99.88 +/- (0.0) %
Average F1-Score: 99.30 +/- (0.0) %
